In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [ ]:
p = 'output/2021_Feb_12_20_08_44_PAMBUH/'

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append(p+'src')

In [ ]:
from pathlib import Path
from PIL import Image
from functools import partial

from tqdm.auto import tqdm
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import albumentations as albu

import  matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import data
# from config import cfg, cfg_init

# cfg_init(str(p +  'cfg.yaml'))
# cfg['PARALLEL']['DDP'] = False
# cfg['DATA']['TRAIN']['PRELOAD'] = False


# datasets = data.build_datasets(cfg)
# dls = data.build_dataloaders(cfg, datasets, pin=True, drop_last=False)
# tdl = dls['TRAIN']
# xb,yb = next(iter(tdl))
# xb.shape, yb.shape

# s = sampler.tif_block_read('./input/hm/test/b2dc8411c.tiff')
# _,_,img = next(iter(s))
# img.shape, img.dtype

In [ ]:
import infer

In [ ]:
foo = infer.get_infer_func(p)

In [ ]:
foo

# Gala backs

In [ ]:
import multiprocessing as mp
import rasterio as rio

In [ ]:
def pad_image(img, pad):
    h, w = img.shape[1], img.shape[2]
    img_pad = np.zeros((3, h+2*pad, w+2*pad)).astype(img.dtype)
    paste(img_pad, img, pad,pad, h, w)
    return img_pad

       
def infer_image(img, do_inference, block_size=512, pad=16):
    img_pad, mask = pad_image(img, pad), np.zeros_like(img).astype(np.uint8)
    print('padded')
    cds = list(generate_block_coords(img.shape[1], img.shape[2], block_size=(block_size,block_size)))
    
    for block_cd in tqdm(cds):
        padded_block_cd = pad_block(*block_cd, pad)
        block = crop(img_pad, *(pad + padded_block_cd))
        if filter_block_out(block): continue
        
        block = infer_blocks([block], do_inference)
        paste_center_crop(mask, block, block_cd)
        
    return mask

In [ ]:
def _count_blocks(dims, block_size=(256, 256)):
    nXBlocks = (int)((dims[0] + block_size[0] - 1) / block_size[0])
    nYBlocks = (int)((dims[1] + block_size[1] - 1) / block_size[1])
    return nXBlocks, nYBlocks

def get_basics_rasterio(name):
    file = rio.open(str(name))
    return file, file.shape, file.count

def generate_block_coords(H, W, block_size=(128,128)):
    h,w = block_size
    nYBlocks = (int)((H + h - 1) / h)
    nXBlocks = (int)((W + w - 1) / w)
    
    nYValid, nXValid = h, w
    for X in range(nXBlocks):
        #if X == nXBlocks - 1: nXValid = W - X * h
        cx = X * h
        nYValid = w
        for Y in range(nYBlocks):
            #if Y == nYBlocks - 1: nYValid = H - Y *w
            cy = Y * w
            yield cy, cx, nYValid, nXValid

            
def get_shift_center(block, h,w): return block.shape[1]//2 -h//2, block.shape[2]//2 -w//2
def pad_block(y,x,h,w, pad): return np.array([y-pad, x-pad, h+pad, w+pad])
            
def crop(src, y,x,h,w): return src[..., y:y+h, x:x+w]
def crop_rio(p, y,x,h,w, bands=(1,2,3)):
    ds = rio.open(p)
    block = ds.read(bands, window=((y,y+h),(x,x+w)), boundless=True)
    ds.close()
    del ds
    return block

def paste(src, block, y,x,h,w):src[..., y:y+h, x:x+w] = block
def paste_center_crop(src, part, block):
    C,H,W = src.shape
    y,x,h,w = block
    
    h, w = min(h, H-y), min(w, W-x)  
    y, x = get_shift_center(part, h,w)
    part = crop(part, y, x, h, w)
    paste(src, part, *block)

def filter_block_out(block): return block.max()<5
def infer_blocks(blocks, do_inference):
    blocks = do_inference(blocks).cpu().numpy()    
    blocks = (255*blocks).astype(np.uint8)
    return blocks
 

def image_q_reader(q, p, cds, pad):
    for block_cd in cds:
        padded_block_cd = pad_block(*block_cd, pad)
        block = crop_rio(p, *(padded_block_cd))
        #print(f'Read block {block.shape, block.dtype}')
        #if filter_block_out(block): continue
        q.put((block, block_cd))
        
def mask_q_writer(q, H, W, total_blocks, root, result):
    do_inference = infer.get_infer_func(root)
    mask = np.zeros((1,H,W)).astype(np.uint8)
    count = 0
    bs = 8
    batch = []
    
    while count < total_blocks:
        try:
            if len(batch) == bs:
                print('Drop batch', len(batch))
                
                #batch = []
                #continue
                
                blocks = [b[0] for b in batch]
                block_cds = [b[1] for b in batch]
                #print(blocks[0].shape, blocks[0].dtype)
                block_masks = infer_blocks(blocks, do_inference)#.squeeze(0)
                [paste_center_crop(mask, block_mask, block_cd) for block_mask, block_cd in zip(block_masks, block_cds)]
                batch = []
            else:
                item = q.get()
                batch.append(item)
                #print('Add batch', len(batch), count, total_blocks, q.qsize())
                count+=1
                q.task_done()
            
        except mp.TimeoutError:
            print("timeout, quit.")
            break
        except Exception as e:
            print(e)
            break
    if batch:
        print('Drop last batch', len(batch))
        batch = []
    
    result['mask'] = mask
    return

def mp_func_wrapper(func, args): return func(*args)
def chunkify(l, n): return [l[i:i + n] for i in range(0, len(l), n)]

In [ ]:

def launch_mpq(p, model_folder, block_size=512, pad=16, num_processes=1, show_tqdm=True, qsize=12):
    m = mp.Manager()
    result = m.dict()
    q = m.Queue(maxsize=qsize)
    _, (H,W), _ = get_basics_rasterio(p)
    cds = list(generate_block_coords(H, W, block_size=(block_size,block_size)))
        
    n_chunks = num_processes#1 + len(cds) // num_processes
    reader_args = [(q, p, part_cds, pad) for part_cds in chunkify(cds, n_chunks)]
    #assert len(reader_args) == num_processes
    reader = partial(mp_func_wrapper, image_q_reader)
    
    total_blocks = len(reader_args)
    writer = partial(mp_func_wrapper, mask_q_writer)

    writer_p = mp.Process(target=writer, args=((q,H,W,total_blocks, model_folder, result),))
    writer_p.start()        
    
    
    if show_tqdm: pbar = tqdm(total=len(reader_args))
    with mp.Pool(num_processes) as p:    
        g = p.imap_unordered(reader, reader_args)
        for _ in g:
            if show_tqdm: pbar.update()    
    
    print('joining')
    writer_p.join()
    print(result)
    mask = result['mask']
    
    print(mask.shape, mask.dtype, mask.max())
    return mask

In [ ]:
img_name = 'input/hm/train/aaa6a05cc.tiff'

In [ ]:
mask = launch_mpq(img_name, p, block_size=1024, pad=128, num_processes=1)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../submission.csv')

In [ ]:
df.columns

In [ ]:
dff = pd.read_csv('../input/hm/sample_submission.csv')

In [ ]:
dff

In [ ]:
# iimg = Image.open('input/cuts1024x05/imgs/0486052bb/000000.png')
# img = np.array(iimg)
# img = np.transpose(img, (2,0,1))
# print(img.shape)

In [ ]:
def inf(img): return img[0]/255.>.5

In [ ]:
mask = infer_image(img, inf, 128,16)
mask.max(), mask.min()

In [ ]:
plt.imshow(mask.astype(np.float32).transpose((1,2,0)))

In [ ]:
#Image.fromarray((255*mask).astype(np.uint8)).resize((512,512))

In [ ]:
import rasterio as rio

In [ ]:
img = rio.open('./input/hm/train/aaa6a05cc.tiff')

In [ ]:
x,y,w,h = -100,-100,256,256
ar = img.read((1,2,3), window=((y,y+h),(x,x+w)), boundless=True)
ar.shape

In [ ]:
plt.imshow(ar.transpose(1,2,0))

In [ ]:
img = rio.open('./input/hm/test/b2dc8411c.tiff').read()

In [ ]:
iimg = img#[...,:9000,:5000]

In [ ]:
mask = infer_image(iimg, foo, 4096, 128)

In [ ]:
mask.shape, mask.dtype, mask.max()

In [ ]:
mask[mask<200] = 0

In [ ]:
def dice_loss(inp, target, eps=1e-6):
    with torch.no_grad():
        a = inp.contiguous().view(-1)
        b = target.contiguous().view(-1)
        intersection = (a * b).sum()
        dice = ((2. * intersection + eps) / (a.sum() + b.sum() + eps)) 
        return dice.item()

In [ ]:
mt = torch.from_numpy(mask)

In [ ]:
gt = rio.open('input/bigmasks/0486052bb.tiff').read()
gt = torch.from_numpy(gt)

In [ ]:
gt.shape, gt.dtype, gt.max()

In [ ]:
dice_loss(mt[0]/255,gt.squeeze())

In [ ]:
mask.max()

In [ ]:
mask[...,:500,:500].max()

In [ ]:
plt.imshow(mask[...,:500,:500].sum(0))

In [ ]:
save_tiff_uint8_single_band(mask[0], 'test.tiff')

In [ ]:
tiff_merge_mask('./input/hm/test/b2dc8411c.tiff', 'test.tiff', 'mtets.tiff')

In [ ]:
def save_tiff_uint8_single_band(img, path):
    assert img.dtype == np.uint8
    if img.max() <= 1. : print(f"Warning: saving tiff with max value is <= 1, {path}")
    h, w = img.shape
    dst = rio.open(path, 'w', driver='GTiff', height=h, width=w, count=1, nbits=1, dtype=np.uint8)
    dst.write(img, 1)
    dst.close()
    del dst

In [ ]:

def tiff_merge_mask(path_tiff, path_mask, path_dst):
    # will use shitload of mem
    img = rio.open(path_tiff).read()
    mask = rio.open(path_mask).read()
    assert mask.max() <= 1 + 1e-6
    img[1,...] = mask*200
    _, h, w = img.shape
    dst = rio.open(path_dst, 'w', driver='GTiff', height=h, width=w, count=3, dtype=np.uint8)
    dst.write(img, [1,2,3]) # 3 bands
    dst.close()
    del dst

In [ ]:
plt.imshow((255*mask.transpose(1,2,0)).astype(np.uint8))

# Tests

## Sampler

In [ ]:
import sampler

In [ ]:
s = sampler.tif_block_read('./input/hm/test/b2dc8411c.tiff', block_size=(1024,1024))
s = iter(s)
_,_,img = next(s)
print(img.shape, img.dtype, img.max())

In [ ]:
Image.fromarray(img.transpose(1,2,0)).resize((256,256))

## Rasterio

In [ ]:
import rasterio as rio

In [ ]:
ds = rio.open('./input/hm/train/aaa6a05cc.tiff')
ds.shape

In [ ]:
i = ds.read()

In [ ]:
i.shape

In [ ]:
x,y = 22750,16100
w,h=2048,2048
img = ds.read([1,2,3], window=((x,x+w),(y,y+h)))
print(img.shape, img.dtype)

In [ ]:
Image.fromarray(img.transpose(1,2,0)).resize((256,256))

In [ ]:
#%%timeit -n 10 -r 10
bb = foo([img]).cpu()
bb.shape, bb.dtype, bb.max()

In [ ]:
plt.imshow(bb[0][0])

In [ ]:
t = callbacks.denorm(dxb[0], mean=cfg.TRANSFORMERS.MEAN, std=cfg.TRANSFORMERS.STD).squeeze()
t = t.squeeze().permute(1,2,0).numpy()
t.shape

Image.fromarray((255.*t).astype(np.uint8))#.resize((512,512))